# Alchool Expenditure Classification

# Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Functions

In [5]:
def check_nulls_dupes(df):
  print(f"Total Amount Null Values: {df.isna().sum()}")
  print(f"Total Amount of Duplicated Values: {df.duplicated().sum()}")

# EDA Analysis

In [2]:
df = pd.read_csv("/Users/maukanmir/Downloads/data.csv")
df

,country,Code,Year,"Other (at home) (USDA, 2018)","Food stores (at home) (USDA, 2018)","Liquor stores (at home) (USDA, 2018)","All other (away from home) (USDA, 2018)","Hotels & motels (away from home) (USDA, 2018)","Restaurants & bars (away from home) (USDA, 2018)",percentage
0,United States,USA,1935,199000000,65000000,305000000,20000000,81000000,964000000,"12,18%"
1,United States,USA,1936,220000000,95000000,435000000,24000000,97000000,1195000000,"3,98%"
2,United States,USA,1937,235000000,113000000,504000000,28000000,109000000,1299000000,"18,67%"
3,United States,USA,1938,227000000,111000000,479000000,26000000,98000000,1246000000,"1,22%"
4,United States,USA,1939,237000000,122000000,517000000,28000000,103000000,1365000000,"4,96%"
...,...,...,...,...,...,...,...,...,...,...
75,United States,USA,2010,20325518116,20728382105,40244994481,10182515190,4721220000,55153264810,"7,30%"
76,United States,USA,2011,22398210076,21802641216,40828786690,10448883481,4774329000,57935787519,"3,43%"
77,United States,USA,2012,23646140574,22462872896,43336680824,10946909055,4882059000,61508031945,"38,63%"
78,United States,USA,2013,22446796526,22921754302,44994457708,11414860599,4898664000,62861475401,"13,24%"


In [3]:
df.dtypes

country                                             object
Code                                                object
Year                                                 int64
Other (at home) (USDA, 2018)                         int64
Food stores (at home) (USDA, 2018)                   int64
Liquor stores (at home) (USDA, 2018)                 int64
All other (away from home) (USDA, 2018)              int64
Hotels & motels (away from home) (USDA, 2018)        int64
Restaurants & bars (away from home) (USDA, 2018)     int64
percentage                                          object
dtype: object

# Check for Nulls and Dupes

In [6]:
check_nulls_dupes(df)

Total Amount Null Values: country                                             0
Code                                                0
Year                                                0
Other (at home) (USDA, 2018)                        0
Food stores (at home) (USDA, 2018)                  0
Liquor stores (at home) (USDA, 2018)                0
All other (away from home) (USDA, 2018)             0
Hotels & motels (away from home) (USDA, 2018)       0
Restaurants & bars (away from home) (USDA, 2018)    0
percentage                                          0
dtype: int64
Total Amount of Duplicated Values: 0
